# INF200 Lecture No J02
### Hans Ekkehard Plesser / NMBU
### 3 June 2020

## Today's topics
- Python
    - Class, static, and private methods
    - Repetition: Mutables as default arguments
- Development
    - A little bit about agile development
    - Shared repositories, branches and pull requests

## Class methods

- Methods usually work on individual objects
- Sometimes, it can be useful to do things at a class level
- Examples
    - count number of instances of a class
    - set parameters that apply to all members of a class
- We can achieve this by writing *class methods*
- A method becomes a class method by adding the `@classmethod` *decorator*
- The `self` argument is replaced by `cls` in class methods

In [1]:
class Truck:
    
    instance_count = 0       # number of trucks
    weight_empty = 1000      # weight of empty truck
    
    @classmethod
    def count_new_truck(cls):
        cls.instance_count += 1
        
    @classmethod
    def num_trucks(cls):
        return cls.instance_count
    
    @classmethod
    def set_weight_empty(cls, we):
        cls.weight_empty = we
        
    def __init__(self, load):
        self._load = load
        self.count_new_truck()
        
    def total_weight(self):
        return self._load + self.weight_empty
    
Truck.set_weight_empty(1500)
trucks = [Truck(load) for load in [100, 500, 1000]]
print("Number of trucks:", Truck.num_trucks())

for truck in trucks:
    print("Total weight:", truck.total_weight())

Number of trucks: 3
Total weight: 1600
Total weight: 2000
Total weight: 2500


Note the following:

- We can access class attributes through `self`
- When counting new trucks, we must make sure that we update the class attribute `instance_count`,  not create an `instance_count` attribute in the instance created. Therefore, we use the *class* method `count_new_truck()`.
- When calling `self.count_new_truck()`, Python automatically makes sure that the class of `self`,  not `self` is passed as parameter `cls`.

### Class methods and inheritance

- The `cls` argument passed to a class method is always the concrete class of the object on which the class method is called

In [2]:
class A:
    _info = None
    
    @classmethod
    def print_info(cls):
        print("Class info:", cls._info)
        
    def display(self):
        print("Displaying ...", end=' ')
        self.print_info()
        
class B(A):
    _info = "This is class B"
    
class C(A):
    _info = "This is class C"
    
class D(C):
    _info = "This is class D"
    
b, c, d = B(), C(), D()

##### Call `print_info()` on instances

In [3]:
b.print_info()
c.print_info()
d.print_info()

Class info: This is class B
Class info: This is class C
Class info: This is class D


##### Call `display()` on instances, which then calls `print_info()`

In [4]:
b.display()
c.display()
d.display()

Displaying ... Class info: This is class B
Displaying ... Class info: This is class C
Displaying ... Class info: This is class D


- `cls._info` always resolves to the `_info` class attribute defined in the concrete class to which the instance belongs.

### Static methods

- Sometimes, it can be useful to have a function in a class that behaves as a normal function, i.e., does not need any access to "self". 
- In some cases, one will define such a function outside the class.
- In other cases, it can be useful to define the function inside the class to show where it belongs logically.
- Static methods are used for this purpose. They are defined using the `@staticmethod` decorator.
- Note that they only get passed the arguments explicitly given in the call, no `self` is inserted anywhere.

In [5]:
import random

class Game:
    
    def __init__(self, seed):
        random.seed(seed)
        self.results = []
        
    def play(self):
        n1 = random.random()
        n2 = random.random()
        n3 = random.random()
        res = self._median(n1, n2, n3)
        self.results.append(res)
        
    @staticmethod    
    def _median(a, b, c):
        return sorted([a, b, c])[1]
    
g = Game(12345)
for _ in range(10):
    g.play()
    
g.results

[0.41661987254534116,
 0.2986398551995928,
 0.1616878239293682,
 0.4329362680099159,
 0.5532210855693298,
 0.412119392939301,
 0.5039353681100375,
 0.18997137872182035,
 0.9674824588798714,
 0.7445300401410346]

### Private methods

- Sometimes, it is useful to define "helper" methods that should be used only by other methods of the same class
- To mark these methods as private, start the method name with `_`, e.g. `_helper(self, ...)`
- Private methods are an *implementation detail*
    - By making a method private, we tell other programmers using our module or class that they should not call this method.
    - This leaves us free to change the method later.
- We can do the same for data attributes. 

## Mutables as default arguments

- Mutables should **never** be used as default arguments
- Reason: The default value is a mutable object created when the function or method is defined. Therefore, all calls of the method will receive **the same mutable object** as argument
- [Example on Python Tutor](http://pythontutor.com/visualize.html#code=class%20A%3A%0A%20%20%20%20def%20__init__%28self,%20data%3D%5B%5D%29%3A%0A%20%20%20%20%20%20%20%20self.data%20%3D%20data%0A%20%20%20%20def%20add%28self,%20new_data%29%3A%0A%20%20%20%20%20%20%20%20self.data.extend%28new_data%29%0A%0Aa%20%3D%20A%28%5B1,%202,%203%5D%29%0Aa.add%28%5B'a',%20'b'%5D%29%0A%20%20%20%20%20%20%20%20%0Ab%20%3D%20A%28%29%0Ab.add%28%5B'c',%20'd'%5D%29%0A%0Ac%20%3D%20A%28%29&cumulative=false&curInstr=0&heapPrimitives=nevernest&mode=display&origin=opt-frontend.js&py=3&rawInputLstJSON=%5B%5D&textReferences=false) (see what happens when `c` is created!)

- Solution: Use `None` as default value and create new mutable in function if necessary

In [6]:
class A:
    def __init__(self, data=None):
        self.data = data if data is not None else []
    def add(self, new_data):
        self.data.extend(new_data)
        
a = A([1, 2, 3])
a.add(['a', 'b'])

b = A()
b.add(['c', 'd'])

c = A()

a.data, b.data, c.data

([1, 2, 3, 'a', 'b'], ['c', 'd'], [])

------------------------------------------

## A little bit on Agile development

- [Agile software development](https://en.wikipedia.org/wiki/Agile_software_development) is a modern (2001-) set of software development methods
- Focus on quick delivery, frequent updates, and flexibility, while maintaining quality
    1. Get it to work
    1. Get it right
    1. Get it fast
- We will use in particular three Agile techniques
    - [Pair programming](https://en.wikipedia.org/wiki/Pair_programming)
    - [Kanban boards](https://en.wikipedia.org/wiki/Kanban_board) (Github project)
    - [Test-driven development](https://en.wikipedia.org/wiki/Test-driven_development) (in a mild version)
- Typical workflow in this project
    - Every morning, review the cards on your board (more often if needed)
        - Any cards "in progress" or "under review" that block further work?
        - Prioritise "to do" cards and select which to start to work on 
        - Make sure you do not have more cards "in progress"/"under review" than you can actually work on at one time
    - Move cards between columns as you progress
    - Create an issue when you
        - identify something that needs to be done
        - discover a bug
    - When you create an issue
        - assign it to a milestone
        - assign it to your project and put it into the "to do" column
    - Develop code in branches and integrate them via pull requests
    
### A real-life example    
For a Github repo of a real-life project using a mild agile approach, see https://github.com/nest/nest-simulator.

## Shared repositories, branches, and pull requests

See videos on the Vimeo channel for examples of the processes described below.

### The basic rule 
**Never force a push**

#### Corrollary

1. Pull changes from repository
1. Merge if necessary (Git/GitKraken will do this automatically for you)
    1. Resolve merge conflicts and commit resolved state
1. Push

#### Exercise 1

Try the following later today!

##### Round 1
- Both of you make changes to `README.md`, one at the top and one at the bottom of the file
- Try pushing
- If you get an error, see Corrollary
- Continue until both your changes are pushed, and both of you have the changes on your computers

##### Round 2
- Begin as in round 1, but make incompatible changes to the same line of `README.md`
- Try pushing, see Corrollary in case of error
- Continue until both your changes are pushed, and both of you have the changes on your computers

### Developing using branches and pull requests

- In larger and long-living projects, code is precious and must be protected from uncontrolled changes.
- Therefore, changes to the master branch need to be controlled.

### Controlled code evolution
- For every change, create a branch
- Develop code in the branch
- Push branch regularly to repo so partner has access, too
- When you think that code is ready for integration to master,
    1. push to repo
    1. go to repo and create a pull request (PR) to master
    1. let your partner review your code
    1. make changes if required and push, the PR is automatically updated
- Once your partner has approved your PR, merge PR into master

### Keeping branches up to date

- Sometimes, branches can live for quite a while
- Meanwhile, `master` will evolve (as PRs are merged in)
- It is then often useful to update the branch with changes in `master`
- To keep the branch up to date with changes in `master`
    1. make sure your branch is checked out
    1. use GitKraken or the git tool of your choice to merge master into your branch
    1. resolve merge conflicts if necessary
    1. push the merged branch to repo
    
### Local and remote branches

- When you create a branch on your computer, it is a *local branch*
- You can only check out and work on local branches
- When you push a branch to the repo, you create a *remote branch*
- By default you local branch will *track* the remote branch
    - You can easily pull changes from and push changes to the remote branch
- Your partner may have to *Fetch* from the repo to see new remote branches you have created
- Once your your partner can see the remote branch in her git tool, she can check it out, creating her own local branch


### Exercise 2

Try this later today!

1. Each of you creates a local branch (choose different names!)
1. Create files `README_<branchname>.md` in your respective branches, fill them with some text
1. Push your branches to the repo
1. Go to Github and create a PR from your branch to master
1. Comment on the PR your partner created
1. Make changes to your own branch in response to your partners comments
1. Once your partner approves your PR, merge it to master
1. Let's say A's PR was merged first. Then B should now merge the changes from master into her PR.
1. Now it is time to merge also B's PR into master, provide A is happy with it.
1. Afterwards, both A and B check out master and pull all changes in master from the repo.


#### Tidy up your shared repository now
Once you have completed the exercises above

- Delete all branches you created during these exercises
- Checkout `master`
- Delete the `README_<branchname>.md` files
- Edit your `README.md` file so that it gives a brief introduction to your team and project.